## Event based

In [1]:
import xarray as xr

In [2]:
#Point of interest for heatwave events
lat = -23.25
lon = 113.5

In [3]:
# opening the input files with heatwave severity data 
thw = xr.open_dataset('/g/data/e14/cp3790/Charuni/Tasmania/aus-coastal-sev.nc').sel(latitude=lat, longitude=lon)
mhw = xr.open_dataset('/g/data/e14/cp3790/Charuni/Tasmania/aus-ocean-sev-2.nc').sel(latitude=lat, longitude=lon)

In [4]:
# using climtas.event.find_events to identify days where severity>1 for a minimum of 3 consecutive days 
thw_events = climtas.event.find_events(thw.severity > 1, min_duration = 3)

In [5]:
ds = xr.open_mfdataset('/g/data/e14/cp3790/Charuni/ERA5-MSL/era5_dailymsl_*.nc').sel(longitude=slice(90, 180), latitude=slice(0, -60))
mslp = ds['dmsl']/100
mslp.attrs['units'] = 'hPa'

In [ ]:
# Creating a new data array into which I subsequently save the mslp values
comp = xr.DataArray(np.random.randint(0, 100, size=(136, 165)),dims=["lat", "lon"], coords=[ds.latitude, ds.longitude])

In [ ]:
for x in ds.latitude:
    for y in ds.longitude:
        mslp_ = mslp.sel(latitude=x, longitude=y)
        mslp_composite = climtas.event.map_events(mslp_, thw_events, lambda x: x.mean().values) #this gives the means of individual events at a grid cell
        event_mean_mslp = mslp_composite.mean() #this gives the mean of all means 
        comp.loc[{"lat":x, "lon":y}] = event_mean_mslp

# expectation: data array with average values at every grid cell which can then be plotted directly 

In [ ]:
comp